### Statistics Calculation and Plotting


In [3]:
# so that it reloads linked source code files automatically before execution
%load_ext autoreload
%autoreload 2

# Local imports
import Manager
import Plotter_ColorSpreads
import Plotter_AltitudeProfiles
import Plotter_Distribution
import Data
from scicolorscales import *

# System imports
import os
import glob
import ipywidgets as w
import threading 
import time

import warnings
warnings.filterwarnings(action='once') # warnings.filterwarnings('ignore')


# progress bar 
ProgressBar = w.IntProgress( value=0, min=0, max=100, description='Progress:', bar_style='success', style={'bar_color': 'RoyalBlue'}, orientation='horizontal')
class ProgressBarRefresher(threading.Thread):
    def __init__(self):
        threading.Thread.__init__(self)
    def run(self):
        while True:
            if Data.Progress > 0: 
                ProgressBar.value = Data.Progress
                ProgressBar.layout.visibility = 'visible'
            time.sleep(5)


######################## Graphical User Interface ########################
def createGUI():
    ## general
    style1 = {'description_width':'170px'}
    style_large = {'description_width':'400px'}
    layout1 = {'width':'850px'}
    layout_small = {'width':'40px'}
    layout_medium = {'width':'180px'}
    layout_large = {'width':'456px'}
    ## the top level visual elements
    MainPanel = w.VBox()    
    MainTab = w.Tab()
    CalcPanel = w.VBox()
    PlotPanel = w.VBox()
    MainPanel.children = [ MainTab ]
    MainTab.children = [ CalcPanel, PlotPanel ]    
    MainTab.set_title(0, 'Calculate')
    MainTab.set_title(1, 'Plot')    
    ## visual elements for calculation
    CalcTypes = list()
    #CalcTypes.append( "JHEISCAT ~ Joule Heating for EISCAT region, latitudes 60-90deg" )
    #CalcTypes.append( "ConvHeatEISCAT ~ Convection Heating for EISCAT region, latitudes 60-90deg" )
    #CalcTypes.append( "PedCondEISCAT ~ Pedersen Conductivity for EISCAT region, latitudes 60-90deg" )
    #CalcTypes.append( "JHnoWindsEISCAT ~ Joule Heating for EISCAT region, latitudes 60-90, ignoring winds" )
    #CalcTypes.append( "PedCondnoWindsEISCAT ~ Pedersen Conductivity for EISCAT region, latitudes 60-90, ignoring winds" )
    #CalcTypes.append( "JHminusWindHeatEISCAT ~ Ohmic minus Wind heating, for EISCAT region, latitudes 60-90" )
    CalcTypes.append( "JH ~ Joule Heating (Ohmic)" )
    CalcTypes.append( "JHminusWindHeat ~ Ohmic minus Wind heating" )
    CalcTypes.append( "PedCond ~ Pedersen Conductivity" )
    CalcTypes.append( "EEX_si ~ Electric Field East" )
    CalcTypes.append( "EEY_si ~ Electric Field North" )
    CalcTypes.append( "ConvHeat ~ Convection heating" )
    CalcTypes.append( "WindHeat ~ Wind heating" )
    TypeOfCalculation_Dropdown = w.Dropdown( options=CalcTypes, description='Calculation: ', style=style1, layout=layout1)
    DistributionNumOfSlots_Textbox = w.Text(value="100", description="Store distribution for each bin with num of slots =", style=style_large, layout=layout_large)
    MLTfrom_box = w.FloatText( value=0, description='MLT from', layout=layout_medium )
    MLTto_box   = w.FloatText( value=24, description='MLT to', layout=layout_medium )
    MLTstep_box = w.FloatText( value=1, description='MLT step', layout=layout_medium )
    ALTfrom_box = w.FloatText( value=100, description='ALT from', layout=layout_medium )
    ALTto_box   = w.FloatText( value=500, description='ALT to', layout=layout_medium )
    ALTstep_box = w.FloatText( value=2, description='ALT step', layout=layout_medium )
    LATfrom_box = w.FloatText( value=-90, description='LAT from', layout=layout_medium )
    LATto_box   = w.FloatText( value=90, description='LAT to', layout=layout_medium )
    LATstep_box = w.FloatText( value=2.5, description='LAT step', layout=layout_medium )
    KPnum_of_bins_box = w.FloatText( value=2, description='#Kp bins', layout=layout_medium )    
    NetCDFpaths = list()
    NetCDFpaths.append("/home/NAS/TIEGCM_DATA_2/TIEGCM_Lifetime_2015_to_2018_JH_QD/TIEGCM*/*AllVars*.nc")
    NetCDFpaths.append("/home/NAS/TIEGCM_DATA_2/TIEGCM_Lifetime_2015_to_2018_JH_QD/TIEGCM*/*001*AllVars*.nc")
    NetCDFpaths.append("/home/NAS/TIEGCM_DATA/tmp/*.nc")
    NetCDFpath_Dropdown = w.Dropdown( options=NetCDFpaths, description='Input files: ', style=style1, layout=layout1)   #NetCDFpath_Dropdown = w.Dropdown( options=["/media/daedalus/8TB/TIEGCM_DATA/TIEGCM*/*AllVars*.nc"], description='Input files: ', style=style1, layout=layout1)
    Calc_Btn = w.Button(description='Calculate',tooltip="Click here to calculate Statistics",)
    Calc_Btn.style.button_color = 'Gold'
    CalcPanel.children += ( w.HBox( [TypeOfCalculation_Dropdown] ), )
    CalcPanel.children += ( NetCDFpath_Dropdown, )
    CalcPanel.children += ( DistributionNumOfSlots_Textbox, )
    CalcPanel.children += ( w.HBox( [MLTfrom_box, MLTto_box, MLTstep_box] ), )
    CalcPanel.children += ( w.HBox( [ALTfrom_box, ALTto_box, ALTstep_box] ), )
    CalcPanel.children += ( w.HBox( [LATfrom_box, LATto_box, LATstep_box] ), )
    CalcPanel.children += ( w.HBox( [KPnum_of_bins_box] ), )
    CalcPanel.children += ( Calc_Btn, )
    ProgressBar.layout.visibility = 'hidden'
    CalcPanel.children += ( ProgressBar, )
    ## visual elements for plotting
    ResultFiles_Dropdown  = w.Dropdown( options=sorted(glob.glob("results/*.nc*")), description='Result files: ', style=style1, layout=layout1)
    ResultFiles2_Dropdown = w.Dropdown( options=[""]+sorted(glob.glob("results/*.nc*")), description='Result files 2: ', style=style1, layout=layout1)
    Plot_Btn = w.Button(description='Plot',tooltip="Click here to plot the results",)
    Plot_Btn.style.button_color = 'YellowGreen'
    PlotPanel.children = [ ResultFiles_Dropdown, ResultFiles2_Dropdown, Plot_Btn ]
    ## Assign event listeners
    def Calc_Btn_Clicked( b ):
        TypeOfCalculation = TypeOfCalculation_Dropdown.value[:TypeOfCalculation_Dropdown.value.rindex('~')].strip()
        Data.setDataParams(MLTfrom_box.value, MLTto_box.value, MLTstep_box.value, LATfrom_box.value, LATto_box.value, LATstep_box.value, ALTfrom_box.value, ALTto_box.value, ALTstep_box.value, KPnum_of_bins_box.value, TypeOfCalculation, int(DistributionNumOfSlots_Textbox.value))
        #print("GUI   ", Data.MLT_min, Data.MLT_max, Data.MLT_duration_of_a_bucket, Data.Alt_min, Data.Alt_max, Data.ALT_distance_of_a_bucket, Data.MagLat_min, Data.MagLat_max, Data.MagLat_degrees_of_a_bucket, Data.num_of_KP_bins, Data.ResultFilename, Data.TypeOfCalculation, Data.KPsequence, Data.ALTsequence, Data.MagLatSequence, Data.MLTsequence )
        if os.path.exists(Data.ResultFilename):
            print("Result file", Data.ResultFilename, "already exists.\nI abort in order to prevent accidental overwrite.")
        else:
            Manager.StartCalculating( NetCDFpath_Dropdown.value, Data.ResultFilename, TypeOfCalculation)
    def Plot_Btn_Clicked( b ):
        '''
        VARS: Ohmic Perpendicular_current_out Hall_conductivity Pedersen_conductivity Hall_current Pedersen_current qDTi_n ,qDTe_i
        Units in xml file (not in plot):
          Ohmic: W/m^3
          Perpendicular_current_out: A/m^2
          Hall_conductivity: S/m
          Pedersen_conductivity: S/m
          currents: A/m^2
        '''
        orbBuckets = None
        if len(ResultFiles2_Dropdown.value) > 0: orbBuckets, orbBinSums, orbBinLens, orbVariableName, orbUnits = Data.LoadResultsCDF(ResultFiles2_Dropdown.value)
        
        simpleResultsFilename = ResultFiles_Dropdown.value[ ResultFiles_Dropdown.value.rindex('/')+1 : ]
        Buckets, BinSums, BinLens, VariableName, Units = Data.LoadResultsCDF(ResultFiles_Dropdown.value)
        
        #Plotter_Distribution.plotDistribution( VariableName, Buckets, SuperTitle=simpleResultsFilename, OrbitBuckets=orbBuckets )
        
        Plotter_AltitudeProfiles.plotAltProfiles( VariableName, Buckets, SuperTitle=simpleResultsFilename )
        #Plotter_ColorSpreads.plotColorSpread_perKpRange( VariableName, Buckets, LogScale=True,  ColorMap="Jet", SuperTitle=simpleResultsFilename  )
        #StatsPlotter.plotColorSpread_AltAxes( VariableName, Buckets, LogScale=True,  ColorMap="Jet", SuperTitle="Mean of "+ VariableName.replace('_',' ')+" ("+Units+")"  )
        #StatsPlotter.plotColorSpread_MLTvsMAGLAT_slice( VariableName, Buckets, LogScale=True,  ColorMap="Jet", SuperTitle="Mean of "+ VariableName.replace('_',' ')+" ("+Units+")"  )
        #StatsPlotter.plotColorSpread_perKpRange( VariableName, Buckets, LogScale=True,  ColorMap="Jet", SuperTitle="Mean of "+ VariableName.replace('_',' ')+" ("+Units+")"  )
        '''
        BucketsOhmic = XML.LoadXMLresults( "Ohmic", "results.Ohmic.xml" )
        BucketsqDTe_i = XML.LoadXMLresults( "qDTe_i", "results.qDTe_i.xml" )
        BucketsqDTi_n = XML.LoadXMLresults( "qDTi_n", "results.qDTi_n.xml" )
        Buckets_A   = init_ResultDataStructure()
        Buckets_B   = init_ResultDataStructure()
        for aKP in KPsequence:
            for anALT in ALTsequence:
                for aMagLat in MagLatSequence:
                    for aMLT in MLTsequence:
                        MeanOhmic = MeanqDTe_i = MeanqDTi_n = 0
                        if BucketsOhmic[(aKP, anALT, aMagLat, aMLT, "Len")] > 0: MeanOhmic = BucketsOhmic[(aKP, anALT, aMagLat, aMLT, "Sum")] / BucketsOhmic[(aKP, anALT, aMagLat, aMLT, "Len")]
                        if BucketsqDTe_i[(aKP, anALT, aMagLat, aMLT, "Len")] > 0: MeanqDTe_i = BucketsqDTe_i[(aKP, anALT, aMagLat, aMLT, "Sum")] / BucketsqDTe_i[(aKP, anALT, aMagLat, aMLT, "Len")]
                        if BucketsqDTi_n[(aKP, anALT, aMagLat, aMLT, "Len")] > 0: MeanqDTi_n = BucketsqDTi_n[(aKP, anALT, aMagLat, aMLT, "Sum")] / BucketsqDTi_n[(aKP, anALT, aMagLat, aMLT, "Len")]
                        Buckets_A[(aKP, anALT, aMagLat, aMLT, "Sum")] = MeanqDTe_i / MeanOhmic
                        Buckets_A[(aKP, anALT, aMagLat, aMLT, "Len")] = 1
                        Buckets_B[(aKP, anALT, aMagLat, aMLT, "Sum")] = MeanqDTi_n / MeanOhmic
                        Buckets_B[(aKP, anALT, aMagLat, aMLT, "Len")] = 1        
        StatsPlotter.plotColorSpread_perKpRange( "qDTe_i/Ohmic", Buckets_A, LogScale=False, ColorMap="Jet", SuperTitle="qDTe_i/Ohmic Means" )
        StatsPlotter.plotColorSpread_perKpRange( "qDTi_n/Ohmic", Buckets_B, LogScale=False, ColorMap="Jet", SuperTitle="qDTi_n/Ohmic Means"   )
        '''
        
        #Buckets, CDFs = XML.LoadXMLresults( VARNAME, ResultFiles_Dropdown.value ) 
        #StatsPlotter.plotColorSpread_AltAxes( VARNAME, Buckets, LogScale=True,  ColorMap="Jet", SuperTitle="Mean of "+VARNAME.replace('_',' ')+" ("+UNITS+")"  )
        #StatsPlotter.plotColorSpread_AltAxes( VARNAME, Buckets, LogScale=False, ColorMap="Jet", SuperTitle="Mean of "+VARNAME.replace('_',' ')+" ("+UNITS+")"  )
        #StatsPlotter.plotColorSpread_AltAxes( VARNAME, Buckets, LogScale=True, ColorMap=allojal, SuperTitle="Mean of "+VARNAME.replace('_',' ')+" ("+UNITS+")"  )
        
        #StatsPlotter.plotColorSpread_perKpRange( "VARNAME", Buckets, LogScale=True,  ColorMap="Jet", SuperTitle=VARNAME+" Means" )
        #StatsPlotter.plotColorSpread_perKpRange( "VARNAME", Buckets, LogScale=False, ColorMap="Jet", SuperTitle=VARNAME+" Means" )
        
        '''
        Buckets = XML.LoadXMLresults( "Hall_conductivity", ResultFiles_Dropdown.value ) 
        StatsPlotter.plotColorSpread_AltAxes( "Hall_conductivity", Buckets, LogScale=True, ColorMap="Jet"  )        
        Buckets = XML.LoadXMLresults( "Pedersen_conductivity", ResultFiles_Dropdown.value ) 
        StatsPlotter.plotColorSpread_AltAxes( "Pedersen_conductivity", Buckets, LogScale=True, ColorMap="Jet"  )
        Buckets = XML.LoadXMLresults( "Hall_current", ResultFiles_Dropdown.value ) 
        StatsPlotter.plotColorSpread_AltAxes( "Hall_current", Buckets, LogScale=True, ColorMap="Jet"  )
        Buckets = XML.LoadXMLresults( "Pedersen_current", ResultFiles_Dropdown.value ) 
        StatsPlotter.plotColorSpread_AltAxes( "Pedersen_current", Buckets, LogScale=True, ColorMap="Jet"  )
        Buckets = XML.LoadXMLresults( "qDTe_i", ResultFiles_Dropdown.value ) 
        StatsPlotter.plotColorSpread_perKpRange( "qDTe_i", Buckets, LogScale=False, ColorMap="Jet"  )
        Buckets = XML.LoadXMLresults( "qDTi_n", ResultFiles_Dropdown.value ) 
        StatsPlotter.plotColorSpread_perKpRange( "qDTi_n", Buckets, LogScale=False, ColorMap="Jet"  )
        '''
        ##
        #Buckets_qDTin = XML.LoadXMLresults( "qDTi_n", ResultFiles_Dropdown.value ) 
        #StatsPlotter.plotColorSpread_perKpRange( "qDTi_n", Buckets_qDTin )
        ##
        #Buckets_qDTei = XML.LoadXMLresults( "qDTe_i", ResultFiles_Dropdown.value ) 
        #StatsPlotter.plotColorSpread_perKpRange( "qDTei", Buckets_qDTin )
        ##
        #Buckets_qDTin_div_Ohmic = ...
        
    #
    Calc_Btn.on_click( Calc_Btn_Clicked )
    Plot_Btn.on_click( Plot_Btn_Clicked )
    ##
    return MainPanel

print( "For EISCAT comparison choose: MLT_15_39_3  LAT_71_78.5_180  ALT_95_150_5 Kp3Bins" )
display( createGUI() )

T = ProgressBarRefresher()
T.start()




The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
For EISCAT comparison choose: MLT_15_39_3  LAT_71_78.5_180  ALT_95_150_5 Kp3Bins


/home/balukid/STATS/Data.py:212: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  DateOfCreation = CDFroot.DateOfCreation
/home/balukid/STATS/Data.py:218: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  MLTs    = CDFroot.variables['MLT'][:]


Opened  results/JH__MLT_15_39_6_LAT_71_78.5_7.5_ALT_95_150_5_Kp3Bins.nc :
   DateOfCreation: 08-06-2021 20:59:22  TypeOfCalculation: JH
   Variable: Joule Heating  Units: W/m3  filled bins: 132 / 132  Num of Kp bins: 3
   MLT: 15.0 - 39.0 step 6.0   Alt.: 95.0 - 150.0 step 5.0   Mag.Lat.: 71.0 - 251.0 step 180




/home/maze/anaconda3/lib/python3.7/site-packages/nbformat/notebooknode.py:4: DeprecationWarning:

Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working

